<a href="https://colab.research.google.com/github/dfvnunes/GTM-Setup/blob/main/Assist%C3%AAncias_por_clube_Casa_Fora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython import get_ipython
from IPython.display import display
# %%
!pip install xlsxwriter==3.0.3
import pandas as pd
import re
import xlsxwriter

# Função para extrair o clube da coluna 'Jogo' com base na posição
def extrair_clube(jogo, posicao, temporada, dicionario_clubes):
    clubes_da_temporada = dicionario_clubes.get(temporada, [])
    # Separar os clubes no nome do jogo usando regex
    clubes_no_jogo = re.findall(r'\b(?:' + '|'.join(clubes_da_temporada) + r')\b', jogo)
    if len(clubes_no_jogo) > posicao:
        return clubes_no_jogo[posicao]
    else:
        return None

# Ler o ficheiro Excel com os clubes por temporada e liga
clubes_por_temporada_file = pd.ExcelFile('Dicionário de clubes por época.xlsm')
dicionario_clubes = {}
for sheet_name in clubes_por_temporada_file.sheet_names:
    df_clubes = clubes_por_temporada_file.parse(sheet_name)
    clubes_da_temporada = df_clubes['Liga 1'].tolist() + df_clubes['Liga 2'].tolist()
    clubes_da_temporada = [str(clube) for clube in clubes_da_temporada if pd.notna(clube)]
    dicionario_clubes[sheet_name] = clubes_da_temporada

# Função para obter a liga do clube
def obter_liga(clube, temporada, dicionario_clubes):
    if clube in dicionario_clubes.get(temporada, []):
        df_liga = pd.read_excel('Dicionário de clubes por época.xlsm', sheet_name=temporada)
        if clube in df_liga['Liga 1'].values:
            return 'Liga 1'
        elif clube in df_liga['Liga 2'].values:
            return 'Liga 2'
    return None

# Ler o ficheiro Excel com os dados dos jogos
excel_file = pd.ExcelFile('Assistencias por Jogo Final.xlsx')

# Criar um dicionário para armazenar as DataFrames de cada época
dataframes_por_epoca = {}

# Iterar pelas folhas do Excel
for sheet_name in excel_file.sheet_names:
    df = excel_file.parse(sheet_name)

    # Extrair clubes da coluna 'Jogo'
    df['Clube Casa'] = df.apply(lambda row: extrair_clube(row['Jogo'], 0, row['Temporada'], dicionario_clubes), axis=1)
    df['Clube Fora'] = df.apply(lambda row: extrair_clube(row['Jogo'], 1, row['Temporada'], dicionario_clubes), axis=1)

    # Criar um dicionário para armazenar as assistências por clube
    assistencias_por_clube = {}

    # Iterar pelas linhas do DataFrame
    for index, row in df.iterrows():
        clube_casa = row['Clube Casa']
        clube_fora = row['Clube Fora']
        espectadores = row['Espectadores']

        # Adicionar assistências para o clube da casa
        assistencias_por_clube.setdefault(clube_casa, {'Assistências Casa': 0, 'Assistências Fora': 0, 'Total por Clube': 0})
        assistencias_por_clube[clube_casa]['Assistências Casa'] += espectadores
        assistencias_por_clube[clube_casa]['Total por Clube'] += espectadores

        # Adicionar assistências para o clube visitante, exceto se o jogo for em casa de um dos "três grandes"
        if clube_casa not in ["SL Benfica", "Sporting CP", "FC Porto"]:  # Nova condição
            assistencias_por_clube.setdefault(clube_fora, {'Assistências Casa': 0, 'Assistências Fora': 0, 'Total por Clube': 0})
            assistencias_por_clube[clube_fora]['Assistências Fora'] += espectadores

    # Criar um DataFrame a partir do dicionário de assistências
    clube_total = pd.DataFrame.from_dict(assistencias_por_clube, orient='index').reset_index()
    clube_total.rename(columns={'index': 'Clube'}, inplace=True)

    # Calcular o total de assistências por clube
    clube_total['Total por Clube'] = clube_total['Assistências Casa'] + clube_total['Assistências Fora']

    # Adicionar coluna "Liga"
    clube_total['Liga'] = clube_total.apply(lambda row: obter_liga(row['Clube'], sheet_name, dicionario_clubes), axis=1)

    # Adicionar coluna "Temporada"
    clube_total['Temporada'] = sheet_name

    # Reordenar as colunas
    clube_total = clube_total[['Liga', 'Temporada', 'Clube', 'Assistências Casa', 'Assistências Fora', 'Total por Clube']]

    # Armazenar a DataFrame no dicionário dataframes_por_epoca
    dataframes_por_epoca[sheet_name] = clube_total

# Exportar as DataFrames para um ficheiro Excel com colunas centradas
with pd.ExcelWriter('assistencias_por_clube_e_epoca.xlsx', engine='xlsxwriter') as writer:
    for epoca, dataframe in dataframes_por_epoca.items():
        # Ordenar o DataFrame pela coluna "Total por Clube" em ordem descendente
        dataframe = dataframe.sort_values(by=['Total por Clube'], ascending=False)
        dataframe.to_excel(writer, sheet_name=epoca, index=False)
        # Centralizar as colunas
        worksheet = writer.sheets[epoca]
        for col_num, col_name in enumerate(dataframe.columns):
            cell_format = writer.book.add_format()
            cell_format.set_align('center')
            worksheet.set_column(col_num, col_num, 15, cell_format)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 4.6 MB/s eta 0:00:00
